# Lakebase + Registry Setup

This notebook uses Databricks Connect and the Databricks SDK to:

- Create a Lakebase (database) instance.
- Create a `shm.agent_registry` schema in Unity Catalog for visibility.
- Create the registry tables inside the Lakebase database.

It authenticates using the Databricks workspace client, so no explicit host/token is needed if your Databricks Connect profile is configured.

In [ ]:
from databricks.connect import DatabricksSession
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.database import DatabaseInstance
from pathlib import Path
import json
import psycopg
import re
import yaml

config_path = Path("config.yaml")
if not config_path.exists():
    config_path = Path("..") / "config.yaml"
if not config_path.exists():
    raise RuntimeError("Missing config.yaml")

config = yaml.safe_load(config_path.read_text(encoding="utf-8")) or {}
if not isinstance(config, dict):
    raise RuntimeError("config.yaml must be a flat dictionary")

CATALOG = config.get("catalog", "shm")
REGISTRY_SCHEMA = config.get("registry_schema", "agent_registry")
LAKEBASE_INSTANCE_NAME = config.get("lakebase_instance_name", "agent-registry")
LAKEBASE_CAPACITY = config.get("lakebase_capacity", "CU_1")
LAKEBASE_RETENTION_DAYS = int(config.get("lakebase_retention_days", 7))

LAKEBASE_DSN = config.get("lakebase_dsn")
LAKEBASE_HOST = config.get("lakebase_host")
LAKEBASE_DB = config.get("lakebase_db", "databricks_postgres")
A2A_BASE_URL = "/a2a"

In [13]:
w = WorkspaceClient()

existing = None
for inst in w.database.list_database_instances():
    if inst.name == LAKEBASE_INSTANCE_NAME:
        existing = inst
        break

if existing:
    instance = existing
else:
    w.database.create_database_instance(
        DatabaseInstance(
            name=LAKEBASE_INSTANCE_NAME,
            capacity=LAKEBASE_CAPACITY,
            retention_window_in_days=LAKEBASE_RETENTION_DAYS,
        )
    )
    instance = None
    for inst in w.database.list_database_instances():
        if inst.name == LAKEBASE_INSTANCE_NAME:
            instance = inst
            break

if not instance:
    raise RuntimeError("Lakebase instance not found after create.")

print(f"Lakebase instance: {instance.name}")
print(f"Read/write endpoint: {instance.read_write_dns}")

Lakebase instance: agent-registry
Read/write endpoint: instance-4abc793a-34e7-49ea-aec3-e35f9ebefb2e.database.azuredatabricks.net


In [14]:
builder = DatabricksSession.builder
spark = builder.serverless().getOrCreate()

spark.sql(f"CREATE CATALOG IF NOT EXISTS {CATALOG}")
spark.sql(f"CREATE SCHEMA IF NOT EXISTS {CATALOG}.{REGISTRY_SCHEMA}")
print(f"Ensured UC schema: {CATALOG}.{REGISTRY_SCHEMA}")

I0000 00:00:1770615039.425591 2229950 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


Ensured UC schema: shm.agent_registry


In [ ]:
from urllib.parse import quote_plus

lakebase_dsn = LAKEBASE_DSN
if not lakebase_dsn:
    ws = WorkspaceClient()
    token = ws.config.oauth_token().access_token
    user = ws.current_user.me().user_name

    host = LAKEBASE_HOST or instance.read_write_dns
    if ":" in host:
        host, port = host.split(":", 1)
    else:
        port = "5432"

    lakebase_dsn = (
        "postgresql://"
        f"{quote_plus(user)}:{quote_plus(token)}@{host}:{port}/{LAKEBASE_DB}"
        "?sslmode=require"
    )

create_sql = f"""
CREATE SCHEMA IF NOT EXISTS {REGISTRY_SCHEMA};

DROP TABLE IF EXISTS {REGISTRY_SCHEMA}.agent_protocol_cards;
DROP TABLE IF EXISTS {REGISTRY_SCHEMA}.agent_versions;
DROP TABLE IF EXISTS {REGISTRY_SCHEMA}.agents;

CREATE TABLE IF NOT EXISTS {REGISTRY_SCHEMA}.agents (
    agent_id TEXT PRIMARY KEY,
    name TEXT,
    description TEXT,
    owner TEXT,
    status TEXT,
    default_version TEXT,
    created_at TIMESTAMPTZ DEFAULT now(),
    updated_at TIMESTAMPTZ DEFAULT now()
);

CREATE TABLE IF NOT EXISTS {REGISTRY_SCHEMA}.agent_versions (
    agent_id TEXT NOT NULL,
    version TEXT NOT NULL,
    api_url TEXT,
    tags JSONB,
    created_at TIMESTAMPTZ DEFAULT now(),
    updated_at TIMESTAMPTZ DEFAULT now(),
    PRIMARY KEY (agent_id, version),
    FOREIGN KEY (agent_id) REFERENCES {REGISTRY_SCHEMA}.agents(agent_id)
);

CREATE TABLE IF NOT EXISTS {REGISTRY_SCHEMA}.agent_protocol_cards (
    agent_id TEXT NOT NULL,
    version TEXT NOT NULL,
    protocol TEXT NOT NULL,
    card_json JSONB,
    updated_at TIMESTAMPTZ DEFAULT now(),
    PRIMARY KEY (agent_id, version, protocol),
    FOREIGN KEY (agent_id, version) REFERENCES {REGISTRY_SCHEMA}.agent_versions(agent_id, version)
);
"""

with psycopg.connect(lakebase_dsn) as conn:
    conn.execute(create_sql)
    conn.commit()

config["lakebase_host"] = LAKEBASE_HOST or instance.read_write_dns
config["lakebase_db"] = LAKEBASE_DB
config["registry_schema"] = REGISTRY_SCHEMA
config_path.write_text(yaml.safe_dump(config, sort_keys=False), encoding="utf-8")

print("Created Lakebase tables.")
print("Updated config.yaml with lakebase_host and lakebase_db.")

I0000 00:00:1770615040.600709 1694118 fork_posix.cc:71] Other threads are currently calling into gRPC, skipping fork() handlers


Created Lakebase tables.
Updated config.yaml with lakebase_host and lakebase_db.


In [ ]:
test_card_path = Path("examples/agent_cards/test_agent_card.json")
test_card = json.loads(test_card_path.read_text(encoding="utf-8"))

agents_seed = [
    {
        "agent_id": "test-agent",
        "name": "Test Agent",
        "description": "Test Agent for validating MCP and A2A flows.",
        "owner": "setup",
        "status": "active",
        "version": "v1",
        "api_url": None,
        "card": {
            **test_card,
            "url": A2A_BASE_URL,
        },
    }
]

insert_agents = f"""
INSERT INTO {REGISTRY_SCHEMA}.agents (
    agent_id, name, description, owner, status, default_version
) VALUES (
    %(agent_id)s, %(name)s, %(description)s, %(owner)s, %(status)s, %(version)s
)
ON CONFLICT (agent_id) DO UPDATE SET
    name = EXCLUDED.name,
    description = EXCLUDED.description,
    owner = EXCLUDED.owner,
    status = EXCLUDED.status,
    default_version = EXCLUDED.default_version,
    updated_at = now();
"""

insert_versions = f"""
INSERT INTO {REGISTRY_SCHEMA}.agent_versions (
    agent_id, version, api_url, tags
) VALUES (
    %(agent_id)s, %(version)s, %(api_url)s, %(tags)s
)
ON CONFLICT (agent_id, version) DO UPDATE SET
    api_url = EXCLUDED.api_url,
    tags = EXCLUDED.tags,
    updated_at = now();
"""

insert_cards = f"""
INSERT INTO {REGISTRY_SCHEMA}.agent_protocol_cards (
    agent_id, version, protocol, card_json
) VALUES (
    %(agent_id)s, %(version)s, %(protocol)s, %(card_json)s
)
ON CONFLICT (agent_id, version, protocol) DO UPDATE SET
    card_json = EXCLUDED.card_json,
    updated_at = now();
"""

In [18]:
agents_seed

[{'agent_id': 'test-agent',
  'name': 'Test Agent',
  'description': 'Test Agent for validating MCP and A2A flows.',
  'owner': 'setup',
  'status': 'active',
  'version': 'v1',
  'mcp_server_url': 'https://<your-workspace>/api/2.0/mcp/<server-id>',
  'card': {'schemaVersion': '1.0',
   'humanReadableId': 'examples/test_agent',
   'agentVersion': '0.1.0',
   'name': 'Test Agent',
   'description': 'Test Agent for registry registration and discovery.',
   'url': '/a2a',
   'authSchemes': [{'scheme': 'none'}],
   'defaultInputModes': ['text'],
   'defaultOutputModes': ['text'],
   'capabilities': {'streaming': False},
   'skills': [{'id': 'test_agent',
     'name': 'Test Agent',
     'description': 'Handles simple test tasks.'}]}}]

In [ ]:
with psycopg.connect(lakebase_dsn) as conn:
    for agent in agents_seed:
        payload = {
            "agent_id": agent["agent_id"],
            "name": agent["name"],
            "description": agent["description"],
            "owner": agent["owner"],
            "status": agent["status"],
            "version": agent["version"],
            "api_url": agent.get("api_url"),
            "tags": json.dumps({"source": "setup"}),
            "protocol": "a2a",
            "card_json": json.dumps(agent["card"]),
        }
        conn.execute(insert_agents, payload)
        conn.execute(insert_versions, payload)
        conn.execute(insert_cards, payload)
    conn.commit()

print(f"Seeded {len(agents_seed)} agent records.")

Seeded 1 agent records.
